In [36]:
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torchvision
import os
import numpy as np
import matplotlib.pyplot as plt
import time    

In [37]:
transform = transforms.Compose([#transforms.Resize((256,256)),  
                                transforms.Grayscale(),		# the code transforms.Graysclae() is for changing the size [3,100,100] to [1, 100, 100] (notice : [channel, height, width] )
                                transforms.ToTensor(),])

#train_data_path = 'relative path of training data set'
train_data_path = './horse-or-human/train'
trainset = torchvision.datasets.ImageFolder(root=train_data_path, transform=transform)
# change the valuse of batch_size, num_workers for your program
# if shuffle=True, the data reshuffled at every epoch 
trainloader = torch.utils.data.DataLoader(trainset, batch_size=1027, shuffle=False, num_workers=1)  

validation_data_path = './horse-or-human/validation'
valset = torchvision.datasets.ImageFolder(root=validation_data_path, transform=transform)
# change the valuse of batch_size, num_workers for your program
valloader = torch.utils.data.DataLoader(valset, batch_size=1027, shuffle=False, num_workers=1)  


trainList = list()
validList = list()
trainLabelList = list()
validLabelList = list()

for i, data in enumerate(trainloader):
    # inputs is the image
    # labels is the class of the image
    inputs, labels = data


    # if you don't change the image size, it will be [batch_size, 1, 100, 100]

    batch_size = inputs.shape[0]

    # if labels is horse it returns tensor[0,0,0] else it returns tensor[1,1,1]
    # change inputs to matrix 10000*batch_size
    for bat_idx in range(batch_size):

        targMat = inputs[bat_idx][0]

        colVec = np.reshape(targMat, (np.product(targMat.shape), 1), 'F')

        if(bat_idx == 0):
            batMat = colVec
        else:
            batMat = np.concatenate((batMat, colVec), axis = 1)         

    # Add ones because of the value b in coefficient
    ones = np.ones((1, batch_size), dtype = int)
    batMat = np.concatenate((batMat, ones))
    trainList.append(batMat)
    trainLabelList.append(labels)
    
# load validation images of the batch size for every iteration
for i, data in enumerate(valloader):

    # inputs is the image
    # labels is the class of the image
    inputs, labels = data

    # if you don't change the image size, it will be [batch_size, 1, 100, 100]
     # if labels is horse it returns tensor[0,0,0] else it returns tensor[1,1,1]

    batch_size = inputs.shape[0]

    # Change Inputs to matrix 10000*batch_size

    for bat_idx in range(batch_size):
        targMat = inputs[bat_idx][0]
        colVec = np.reshape(targMat, (np.product(targMat.shape), 1), 'F')

        if(bat_idx == 0):
            batMat = colVec
        else:
            batMat = np.concatenate((batMat,colVec), axis = 1)
        
    # Add ones because of the value b in coefficient
    ones = np.ones((1, batch_size), dtype = int)
    batMat = np.concatenate((batMat, ones))
    validList.append(batMat)
    validLabelList.append(labels)

In [51]:
#Initialize Coef to Zeros
w_0 = np.zeros((4,10001), dtype = float)
w_1 = np.zeros((3, 4), dtype = float)
w_2 = np.zeros((1, 3), dtype = float)

#Set Learning Rate
lrnRate = 0.002

# Set Loss Lists
lrnLoss = list()
valLoss = list()

# Set Accurate Lists
lrnAcc = list()
valAcc = list()

# set Elapsed time Lists
elapTime = list()


epoch = -1
lrnAccRate = 0

while(lrnAccRate < 0.85):
    epoch += 1
    # load training images of the batch size for every iteration
    
    #Set Sum of Derivatives to 0
    sumDw_0 = np.zeros((10001,4), dtype = float)
    
    #Set Sum of Loss to 0
    sumL = 0
    
    #Set Sum of Cor to 0
    cor = 0
    
    start_time = time.time()

    for i, a_0 in enumerate(trainList):
        
        batch_size = a_0.shape[1]


        # Start Regression Calculation
        z_0 = np.dot(w_0, a_0)                                          
        a_1 = 1/(1 + np.exp(-z_0))                                       
        
        z_1 = np.dot(w_1, a_1)
        a_2 = 1/(1 + np.exp(-z_1))
        
        z_2 = np.dot(w_2, a_2)
        a_3 = 1/(1 + np.exp(-z_2))
        
        
    
        dz_2 = np.subtract(a_3, trainLabelList[i])  
        dw_2 = np.dot(dz_2, a_2.T)
        w_2 -= lrnRate * dw_2
        
        da_2 = np.dot(w_2.T, dz_2)
        dz_1 = da_2 * a_2 * (1 - a_2) 
        dw_1 = np.dot(dz_1, a_1.T)
        w_1 -= lrnRate * dw_1
        
        da_1 = np.dot(w_1.T, dz_1)
        dz_0 = da_1 * a_1 * (1 - a_1)
        dw_0 = np.dot(dz_0, a_0.T)
        w_0 -= lrnRate * dw_0
        
        
        
        
        
 
        
        
        # Calculate Total Loss
        a_3 = torch.from_numpy(a_3)                                             #change ndarray to tensor
        dLabels = trainLabelList[i].double()                                   #change tensor type to double
        L = -(dLabels) * np.log(a_3) - (1-dLabels) * np.log(1-a_3)      
        sumL += L.sum()
        
        
        
        # Calculate Accuracy
        

        for batIdx in range(batch_size):

            if(a_3[0][batIdx] <= 0.5 and trainLabelList[i][batIdx] == 0):
                cor += 1

            if(a_3[0][batIdx] > 0.5 and trainLabelList[i][batIdx] == 1):
                cor += 1
            
        
    
    totalDataNum = len(trainloader.dataset)
        
    # Calculate dLossdCoef
    
    

    # Update coefs using derivatives
    
    
    elapsed_time = time.time() - start_time
    elapTime.append(elapsed_time)


    # Calculate TotalLoss
    sumL /= totalDataNum

    lrnLoss.append(sumL)
    
    
    # Calculate Accuracy
    
    lrnAccRate = cor/totalDataNum
    lrnAcc.append(lrnAccRate)
    
                


    # Set Sum Of Valid Loss to 0
    sumVL = 0
    # Set Sum of Valid Cor to 0
    vCor = 0
    
    # load validation images of the batch size for every iteration
    for i, a_0 in enumerate(validList):
        
        
        batch_size = batMat.shape[1]
        
        # Start Calculate Loss 
        z_0 = np.dot(w_0, a_0)
        a_1= 1/(1+np.exp(-z_0))
        
        z_1 = np.dot(w_1, a_1)
        a_2= 1/(1+np.exp(-z_1))
        
        z_2 = np.dot(w_2, a_2)
        a_3= 1/(1+np.exp(-z_2))
        
        
        a_3 = torch.from_numpy(a_3)
        dLabels = validLabelList[i].double()
        L = -(dLabels) * np.log(a_3) - (1-dLabels) * np.log(1-a_3)
        sumVL += L.sum()

        
        # Calculate Accuracy
        
        for batIdx in range(batch_size):
            if(a_3[0][batIdx] <= 0.5 and validLabelList[i][batIdx] == 0):
                vCor += 1
                
            if(a_3[0][batIdx] > 0.5 and validLabelList[i][batIdx] == 1):
                vCor += 1
    
    totalValDataNum = len(valloader.dataset)
    
    # CalCulate Total Loss
    sumVL /= totalValDataNum
    valLoss.append(sumVL)
    
    
    # Calculate Accuracy
    vAcc = vCor/totalValDataNum
    valAcc.append(vAcc)
    
    
    print("epoch : ",epoch,',', "lrnCor : ", cor, "valCor : ", vCor, "elapsed time : ", elapsed_time)
    
    

(3, 1027)
epoch :  0 , lrnCor :  500 valCor :  128 elapsed time :  0.06604599952697754
(3, 1027)
epoch :  1 , lrnCor :  527 valCor :  128 elapsed time :  0.043910980224609375
(3, 1027)
epoch :  2 , lrnCor :  527 valCor :  128 elapsed time :  0.04123497009277344
(3, 1027)
epoch :  3 , lrnCor :  527 valCor :  128 elapsed time :  0.04424619674682617
(3, 1027)
epoch :  4 , lrnCor :  527 valCor :  128 elapsed time :  0.043254852294921875
(3, 1027)
epoch :  5 , lrnCor :  527 valCor :  128 elapsed time :  0.04179096221923828
(3, 1027)
epoch :  6 , lrnCor :  527 valCor :  128 elapsed time :  0.04171919822692871
(3, 1027)
epoch :  7 , lrnCor :  527 valCor :  128 elapsed time :  0.04902291297912598
(3, 1027)
epoch :  8 , lrnCor :  527 valCor :  128 elapsed time :  0.04833579063415527
(3, 1027)
epoch :  9 , lrnCor :  527 valCor :  128 elapsed time :  0.045929908752441406
(3, 1027)
epoch :  10 , lrnCor :  527 valCor :  128 elapsed time :  0.054574012756347656
(3, 1027)
epoch :  11 , lrnCor :  527 

KeyboardInterrupt: 

In [ ]:
plt.plot(elapTime, color = 'black', label = "ElapsedTime")
plt.xlabel("Iteration")
plt.ylabel("time")
plt.legend()



In [ ]:
#Plotting Loss

fig, axs = plt.subplots(2,1, figsize = (10,4))


axs[0].plot(lrnLoss, color = 'red', label = "TrainingLoss")


axs[1].plot(valLoss, color = 'blue', label = "ValidationLoss")

axs[0].set(ylabel = 'Loss')
axs[1].set(xlabel = 'Iteration', ylabel = 'Loss')

axs[0].legend()
axs[1].legend()

In [ ]:
#Plotting Accuracy
fig, axs = plt.subplots(2,1, figsize = (10,5))


axs[0].plot(lrnAcc, color = 'orange', label = "TrainingAccuracy")


axs[1].plot(valAcc, color = 'green', label = "ValidationAccuracy")

axs[0].set(ylabel = 'Accuracy')
axs[1].set(xlabel = 'Iteration', ylabel = 'Accuracy')

axs[0].legend()
axs[1].legend()

In [ ]:
#Plotting Altogether

plt.figure(figsize = (20,10))
plt.title('Altogether')
plt.xlabel('Iteration')

plt.plot(lrnLoss, color = 'red', label = 'TrainingLoss')
plt.plot(valLoss, color = 'blue', label = "ValidationLoss")
plt.plot(lrnAcc, color = 'orange', label = "TrainingAccuracy")
plt.plot(valAcc, color = 'green', label = "ValidationAccuracy")

plt.legend()






In [ ]:
print("  Dataset   |   Loss   | Accuracy")

print("  Training  | %.4f | %.4f" % (lrnLoss[-1], lrnAcc[-1]))
print(" Validation| %.4f | %.4f" % (valLoss[-1], valAcc[-1]))

In [52]:
a = np.array([[1,2,3],
            [4,5,6],
            [7,8,9]])
print(a)

print(1-a)
print(a * (1-a))


[[1 2 3]
 [4 5 6]
 [7 8 9]]
[[ 0 -1 -2]
 [-3 -4 -5]
 [-6 -7 -8]]
[[  0  -2  -6]
 [-12 -20 -30]
 [-42 -56 -72]]
